In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.3.5'

In [3]:
from sqlalchemy import create_engine

#engine = create_engine("sqlite://", echo=True) #echo : 내 행동에 대해 항상 loggin하라는 의미
engine = create_engine("sqlite:///:memory:", echo=True)
#engine = create_engine("sqlite:///test.db", echo=True)

print(engine)

Engine(sqlite:///:memory:)


In [4]:
type(engine)

sqlalchemy.engine.base.Engine

In [6]:
#metadata에서 상속받아서 생성
#MetaData.tables <- 테이블에 뭐가 들어있는지 보려면

from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()
users = Table('users', metadata,
             Column('id', Integer, primary_key = True),
              Column('name', String), #string은 VARCHAR로 바꿔서 넘김 (sql에는 string type 없음)
              Column('fullname', String),
             )

addresses = Table('addresses', metadata,
                 Column('id', Integer, primary_key = True),
                 Column('user_id', None, ForeignKey('users.id')),
                 Column('email_address', String, nullable = False),
                 )

metadata.create_all(engine)

2019-07-09 11:12:28,412 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:12:28,413 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:12:28,414 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:12:28,414 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:12:28,416 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:12:28,416 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:12:28,417 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:12:28,418 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-07-09 11:12:28,419 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:12:28,420 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

In [9]:
insert = users.insert()
print(insert) #named 형태로 쿼리가 날아가는 형태로 파라미터가 만들어진다

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [10]:
insert = users.insert().values(name='kim', fullname = 'Anonymous, Kim')
print(insert)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [12]:
insert.compile().params

{'name': 'kim', 'fullname': 'Anonymous, Kim'}

In [13]:
conn = engine.connect()
conn

In [14]:
insert.bind = engine
str(insert)

result = conn.execute(insert)

result.inserted_primary_key

2019-07-09 11:32:15,909 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:32:15,910 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 11:32:15,911 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

In [15]:
insert = users.insert()
result = conn.execute(insert, name = "lee", fullname = "Unknown, Lee")
result.inserted_primary_key

2019-07-09 11:32:53,253 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:32:53,254 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2019-07-09 11:32:53,254 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [16]:
conn.execute(addresses.insert(), [
    {"user_id":1, "email_address":"anonymous.kim@gmail.com"},
    {"user_id":2, "email_address":"unknown,lee@gmail.com"}
]) # 여러 개의 데이터를 한 번에 넣을 수 있음

2019-07-09 11:35:18,634 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:35:18,635 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@gmail.com'), (2, 'unknown,lee@gmail.com'))
2019-07-09 11:35:18,635 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
from sqlalchemy.sql import select

query = select([users])
result = conn.execute(query) #result가 임시 커서를 생성

#result에서 패치시키면 스칼라값으로 나오는 문제

for row in result:
    print(row)

2019-07-09 11:42:14,810 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:42:14,811 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'Unknown, Lee')


In [18]:
result = conn.execute(select([users.c.name, users.c.fullname])) #users의 특정 culumn들

for row in result:
    print(row)

2019-07-09 11:42:38,782 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-07-09 11:42:38,783 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Anonymous, Kim')
('lee', 'Unknown, Lee')


In [19]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [20]:
cur = conn.execute(select([users]))
cur.fetchall() #list로 받아온다

2019-07-09 11:45:36,046 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:45:36,047 INFO sqlalchemy.engine.base.Engine ()


[(1, 'kim', 'Anonymous, Kim'), (2, 'lee', 'Unknown, Lee')]

In [21]:
#where절 어떻게?
#join 어떻게?
from sqlalchemy import and_, or_, not_

print(users.c.id == addresses.c.user_id)
print(users.c.id == 1)
print((users.c.id == 1).compile().params)
print(or_(users.c.id == addresses.c.user_id, users.c.id == 1))
print(and_(users.c.id == addresses.c.user_id, users.c.id == 1))
print(and_(or_(users.c.id == addresses.c.user_id, users.c.id == 1),
          addresses.c.email_address.like("a%")))

users.id = addresses.user_id
users.id = :id_1
{'id_1': 1}
users.id = addresses.user_id OR users.id = :id_1
users.id = addresses.user_id AND users.id = :id_1
(users.id = addresses.user_id OR users.id = :id_1) AND addresses.email_address LIKE :email_address_1


In [22]:
result = conn.execute(select([users]).where(users.c.id==1))
for row in result:
    print(row)

2019-07-09 13:16:30,648 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:16:30,649 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'kim', 'Anonymous, Kim')


In [23]:
result = conn.execute(select([users, addresses]).where(users.c.id==addresses.c.user_id)) #crossjoin
for row in result:
    print(row)

2019-07-09 13:17:06,847 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:17:06,848 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@gmail.com')
(2, 'lee', 'Unknown, Lee', 2, 2, 'unknown,lee@gmail.com')


In [26]:
result = conn.execute(select([users.c.id, users.c.fullname, addresses.c.email_address]).where(users.c.id==addresses.c.user_id))
for row in result:
    print(row)

2019-07-09 13:21:29,482 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:21:29,483 INFO sqlalchemy.engine.base.Engine ()
(1, 'Anonymous, Kim', 'anonymous.kim@gmail.com')
(2, 'Unknown, Lee', 'unknown,lee@gmail.com')


In [29]:
from sqlalchemy import join

print(users.join(addresses)) d#밑의 구문과 같은 의미
print(users.join(addresses, users.c.id == addresses.c.user_id)) #이미 pk랑 fk를 설정했기 때문에 on절이 필요없음

users JOIN addresses ON users.id = addresses.user_id
users JOIN addresses ON users.id = addresses.user_id


In [31]:
query = select([users.c.id, users.c.fullname, addresses.c.email_address]).select_from(users.join(addresses))
result = conn.execute(query).fetchall()
for row in result:
    print(row)

2019-07-09 13:25:26,723 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 13:25:26,724 INFO sqlalchemy.engine.base.Engine ()
(1, 'Anonymous, Kim', 'anonymous.kim@gmail.com')
(2, 'Unknown, Lee', 'unknown,lee@gmail.com')


In [ ]:
#테이블변경은 처음 metadata만들 때만

In [33]:
metadata.clear()

In [34]:
metadata.tables

immutabledict({})

In [35]:
newengine = create_engine("sqlite:///createdb.db", echo=True)

In [41]:
artist = Table("Artist", metadata,
              Column("id", Integer, primary_key = True),
              Column("name", String, nullable = False),
              extend_existing=True)

album = Table("Album", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable = False),
             Column("artist_id", Integer, ForeignKey("Artist.id")),
             extend_existing = True)

genre = Table("Genre", metadata,
             Column("id", Integer, primary_key = True),
             Column("name", String, nullable = False),
             extend_existing=True)

track = Table("Track", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable = False),
             Column("length", Integer, nullable = False),
             Column("rating", Integer, nullable = False),
             Column("count", Integer, nullable = False),
             Column("album_id", Integer, ForeignKey("Album.id")),
             Column("genre_id", Integer, ForeignKey("Genre.id")),
             extend_existing=True)

In [42]:
metadata.create_all(newengine)

2019-07-09 13:39:20,480 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 13:39:20,480 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:20,482 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 13:39:20,482 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:20,483 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 13:39:20,484 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:20,484 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 13:39:20,485 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:20,486 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2019-07-09 13:39:20,487 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:20,488 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:39:20,489 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Genre" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT N

In [44]:
tables = metadata.tables
for table in tables:
    print(table)
    
for table in newengine.table_names():
    print(table)

Artist
Album
Track
Genre
2019-07-09 13:44:59,302 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 13:44:59,303 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


In [45]:
conn = newengine.connect()

In [46]:
conn.execute(artist.insert(), [
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])
conn.execute(album.insert(), [
    {"title":"IV", "artist_id":1},
    {"title":"Who Made Who", "artist_id":2}
])
conn.execute(genre.insert(), [
    {"name":"Rock"},
    {"name":"Metal"}
])

2019-07-09 14:05:38,443 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 14:05:38,444 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2019-07-09 14:05:38,445 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 14:05:38,447 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 14:05:38,447 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2019-07-09 14:05:38,448 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 14:05:38,450 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 14:05:38,451 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2019-07-09 14:05:38,452 INFO sqlalchemy.engine.base.Engine COMMIT


TypeError: 'Insert' object is not subscriptable

In [49]:
conn.execute(track.insert(), [
    {"title":"Black Dog", "rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"Stairway", "rating":5, "length":482, "count":0, "album_id":1, "genre_id":1},
    {"title":"About to rock", "rating":5, "length":313, "count":0, "album_id":2, "genre_id":2},
    {"title":"Who Made Who", "rating":5, "length":297, "count":0, "album_id":2, "genre_id":2}
])

2019-07-09 14:12:18,613 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 14:12:18,613 INFO sqlalchemy.engine.base.Engine (('Black Dog', 297, 5, 0, 1, 1), ('Stairway', 482, 5, 0, 1, 1), ('About to rock', 313, 5, 0, 2, 2), ('Who Made Who', 297, 5, 0, 2, 2))
2019-07-09 14:12:18,615 INFO sqlalchemy.engine.base.Engine COMMIT


In [50]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [51]:
result = conn.execute(track.select().select_from(track.join(album)))

2019-07-09 14:12:40,894 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2019-07-09 14:12:40,895 INFO sqlalchemy.engine.base.Engine ()


In [52]:
for row in result.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [53]:
sesult = conn.execute(track.select().select_from(track.join(album)).where(album.c.id==1))
for row in result.fetchall():
    print(row)

2019-07-09 14:13:50,422 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id 
WHERE "Album".id = ?
2019-07-09 14:13:50,423 INFO sqlalchemy.engine.base.Engine (1,)


In [57]:
print(track.join(album))
print(track.join(album).join(artist))
print(track.join(album).join(artist))
print(track.join(album).join(genre).join(artist))

result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(track.join(album).join(genre).join(artist)))

for row in result.fetchall():
    print(row)
    
result = conn.execute(track
                    .select()
                    .select_from(track.join(album).join(genre).join(artist))
                    .where(and_(genre.c.id==1, artist.c.id==1,)))
for row in result.fetchall():
    print(row)

"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 14:16:53,815 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 14:16:53,815 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Rock', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')
2019-07-09 14:16:53,818 INFO sqlalchemy.engine.base.Engine SEL

In [58]:
conn.close()
metadata.clear()

In [59]:
tables  = metadata.tables
for table in tables:
    print(table)

In [60]:
metadata.tables

immutabledict({})

In [61]:
for table in newengine.table_names():
    print(table)

2019-07-09 14:20:49,065 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:20:49,066 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


위 두 개의 실행 결과가 다르다.  
metadata의 tables는 없음.  
엔진에는 우리가 생성했던 네 개의 테이블 존재.  
--> 지금은 메모리 상에 올라온 인스턴트 등은 없고 dbms에서 관리하는 db에서만 데이터가 있는 상황

In [63]:
conn = newengine.connect()

metadata = Metadata(bind = newengine, reflect = True)
metadata.reflect(bind = newengine)

for row in metadata.tables:
    print(row)

NameError: name 'Metadata' is not defined

In [64]:
from sqlalchemy.ext.declarative import declarative_base
base = declarative_base()

In [68]:
class User(base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column("passwd", String)
    
    def __repr__(self):
        return "<T'User(name = ''%s', fullname = '%s', password = %s)>" % (self.name, self.fullname, self.password)

In [69]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('passwd', String(), table=<users>), schema=None)

In [70]:
base.metadata.create_all(newengine) #base에 정의되어있는 상속받았던 class들을 새롭게 create

2019-07-09 14:34:11,497 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 14:34:11,498 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:34:11,499 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	passwd VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 14:34:11,500 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:34:11,502 INFO sqlalchemy.engine.base.Engine COMMIT


In [71]:
kim = User(name = "kim", fullname = "anonymous, Kim", password = "kimbap heaven")
#User에 대한 instance
#users에 대한 하나의 record
print(kim)
print(kim.id)

<T'User(name = ''kim', fullname = 'anonymous, Kim', password = kimbap heaven)>
None


In [72]:
User.__mapper__

<Mapper at 0x111540128; User>

In [75]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind = newengine) 
#데이터베이스 그리고 베이스를 상속받아 만든 오프젝트들 간에 왔다갔다해주는 역할
session = Session()

In [77]:
session.add(kim)

session.add_all([
    User(name = 'lee', fullname = "unknown, Lee", password = "123456789a"),
    User(name = "park", fullname = "nobody, Park", password = "Parking in Park")
])
#pending
#mapper를 통해서 연관된 클래스에 있는 애들의 instance들을 table에 넣을 준비 함

In [78]:
session.commit()

2019-07-09 14:47:17,190 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:47:17,192 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:47:17,193 INFO sqlalchemy.engine.base.Engine ('kim', 'anonymous, Kim', 'kimbap heaven')
2019-07-09 14:47:17,194 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:47:17,194 INFO sqlalchemy.engine.base.Engine ('lee', 'unknown, Lee', '123456789a')
2019-07-09 14:47:17,195 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:47:17,195 INFO sqlalchemy.engine.base.Engine ('park', 'nobody, Park', 'Parking in Park')
2019-07-09 14:47:17,197 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:47:17,197 INFO sqlalchemy.engine.base.Engine ('lee', 'unknown, Lee', '123456789a')
2019-07-09 14:47:17,198 INFO sqlalchemy.engine.ba

In [79]:
choi = User(name = "choi", fullname = "anonymous, Kim", password = "kimbap heaven")
session.add(choi)

In [80]:
session.dirty #모든 session에 대해 변경 여부 체크

IdentitySet([])

In [81]:
session.is_modified(choi) # 단일 instance에 대해서 변경 여부 체크
#원래는 is_modified == True 이면 .dirty 에 뭐가 있어야 하는데 지금 오류나고 있는듯???

True

In [82]:
session.commit()

2019-07-09 14:49:04,047 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:49:04,048 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:49:04,049 INFO sqlalchemy.engine.base.Engine ('choi', 'anonymous, Kim', 'kimbap heaven')
2019-07-09 14:49:04,050 INFO sqlalchemy.engine.base.Engine COMMIT


In [83]:
session.dirty

IdentitySet([])

In [84]:
session.is_modified(choi)

False

In [85]:
kim.password = "password"
session.dirty #뭔가 뜨면 sink가 안 맞은 것 --> 뭐라도 있으면 반영하기 위해 commit해야함

2019-07-09 14:49:35,350 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:49:35,351 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.id = ?
2019-07-09 14:49:35,352 INFO sqlalchemy.engine.base.Engine (1,)


IdentitySet([<T'User(name = ''kim', fullname = 'anonymous, Kim', password = password)>])

In [86]:
session.is_modified(kim)

True

In [87]:
kim.id

1

In [91]:
for row in session.query(User):
    print(type(row)) #<class '__main__.User'> ... class이기 때문에 .id, .name 등으로 접근 가능
    print(row.id, row.name, row.fullname, row.password)

2019-07-09 15:10:41,516 INFO sqlalchemy.engine.base.Engine SELECT users.passwd AS users_passwd, users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-07-09 15:10:41,517 INFO sqlalchemy.engine.base.Engine ()
<class '__main__.User'>
1 kim anonymous, Kim password
<class '__main__.User'>
2 lee unknown, Lee 123456789a
<class '__main__.User'>
3 park nobody, Park Parking in Park
<class '__main__.User'>
4 lee unknown, Lee 123456789a
<class '__main__.User'>
5 park nobody, Park Parking in Park
<class '__main__.User'>
6 choi anonymous, Kim kimbap heaven


In [92]:
# python operator
for row in session.query(User.id, User.fullname).filter(User.name == "lee"):
    print(type(row))
    print(row.id, row.fullname)
    
# sql expression
for row in session.query(User.id, User.fullname).filter_by(name = "lee"):
    print(type(row))
    print(row.id, row.fullname)

2019-07-09 15:12:36,280 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2019-07-09 15:12:36,281 INFO sqlalchemy.engine.base.Engine ('lee',)
<class 'sqlalchemy.util._collections.result'>
2 unknown, Lee
<class 'sqlalchemy.util._collections.result'>
4 unknown, Lee
2019-07-09 15:12:36,283 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2019-07-09 15:12:36,284 INFO sqlalchemy.engine.base.Engine ('lee',)
<class 'sqlalchemy.util._collections.result'>
2 unknown, Lee
<class 'sqlalchemy.util._collections.result'>
4 unknown, Lee


In [94]:
class Artist(base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
class Track(base):
    __tablename__ = "Track"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey("Album.id"))
    genre_id = Column(Integer, ForeignKey("Genre.id"))

In [96]:
result = session.query(Track.title, Album.title, Genre.name, Artist.name).select_from(Track).join(Album).join(Genre).join(Artist).all()

for row in result:
    print(row)
    
songList = [dict(Track = row[0], Album = row[1], Genre = row[2], Artist = row[3]) for row in result]

songList

2019-07-09 15:21:45,431 INFO sqlalchemy.engine.base.Engine SELECT "Track".title AS "Track_title", "Album".title AS "Album_title", "Genre".name AS "Genre_name", "Artist".name AS "Artist_name" 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 15:21:45,431 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Rock', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


[{'Track': 'Black Dog',
  'Album': 'IV',
  'Genre': 'Rock',
  'Artist': 'Led Zepplin'},
 {'Track': 'Stairway',
  'Album': 'IV',
  'Genre': 'Rock',
  'Artist': 'Led Zepplin'},
 {'Track': 'About to rock',
  'Album': 'Who Made Who',
  'Genre': 'Metal',
  'Artist': 'AC/DC'},
 {'Track': 'Who Made Who',
  'Album': 'Who Made Who',
  'Genre': 'Metal',
  'Artist': 'AC/DC'}]

기본적으로 쿼리를 날리면 스칼라로 받아옴. 튜플의 형태로 하나만 받아온다. (.one()) 만약 쿼리 날려서 selectall해서 다 얻고 싶다면 마지막에 .all()하면 리스트로 받아온다.

In [5]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
base = declarative_base()
from sqlalchemy.orm import sessionmaker, relationship

class Artist(base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    albumList = relationship("Album", back_populates = "artist")
    
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
    artist = relationship("Artist", back_populates = "albumList", uselist = False) #일대다관계
    trackList = relationship("Track", back_populates = "album")
    
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    trackList = relationship("Track", back_populates = "genre")
    
class Track(base):
    __tablename__ = "Track"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey("Album.id"))
    genre_id = Column(Integer, ForeignKey("Genre.id"))
    
    album = relationship("Album", back_populates = "trackList", uselist = False)
    genre = relationship("Genre", back_populates = "trackList", uselist = False)

regular expression : 속도가 가장 빠름  
쉘 스크립트를 짤 때도 강력하게 동작.  
자연어(인공어와 다르게)의 패턴을 찾기가 힘든데 그 패턴을 어떻게든 만들어보려고 하는 노력 중 일환.  
search pattern!  
줄여서 RE  
주어진 텍스트로부터 (unstructured data) 원하는 패턴이 튀어나오도록.  
structured data로 전처리.  
searching, relpacing, parsing!
input에 의해 스트링이 들어오면 특정한 패턴을 찾거나 특정한 패턴에 맞는 애들을 replace시키거나  
엄청나게 많은 텍스트 데이터를 처리하는 데에 속도가 빠름!  
정확하게 말하면 유닉스 리눅스를 사용하여 이걸 할 때 속도가 제일 빠르다.  
한글 같은 경우 특히 이걸 써야 함.  
한글은 표면적인 형태가 다가 아니다.  
얘를 줘 봤자 학습 제대로 안됨.  
예제 대부분 영어. 그러나 한국말 처리 또한 관건.  
영어 - 하나의 어절 자체가 그 자체로 한 단어. -> sequence to sequence model에 잘 맞음.  
조사 처리를 제대로 안 해주면 어근(말의 중심, 시작점)을 못 찾기 때문에  
음가, 음절, 음소, 단어, 형태소 모두 다름.  
이걸 알아야 NLP 또는 NLP를 이용한 음성인식 가능  
주로 search하는 데에 사용.  
특히 웹에서도 많이 사용 - validation을 위한 용도.  
프론트를 하든 백을 하든 정규식은 항상 들어감.  
이메일 체크, 도메인 체크, 전화번호 체크.  
유일한 단점 : learning curve 존재...처음 진입하기가 힘들다.  
quantification을 이용해서 수량에 대해 할 수 있음. 예) 최소 몇 번 반복, a가 몇 번 반복.  
원하는 패턴을 찾았으면 grouping해줄 수 있음. (html parsing할 때 많이 사용.).  
. : any character. 키보드를 사용해서, 또는 문자표에서 사용할 수 있는 문자 하나.  
a.b : a + 아무 글자 + b  
* : 반복  
ab*c : b가 0번 이상 반복  
?(question) : boolean  


문장이나 문단에 적용하긴 좀 힘들고 단어레벨에서 유용